In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 1. Clone the repository
    !git clone https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    %cd gemini-fullstack-langgraph-quickstart/backend

    # 2. Prepare Environment (Resolving Conflicts)
    import sys
    print("Uninstalling conflicting pre-installed packages...")
    !pip uninstall -y google-ai-generativelanguage tensorflow grpcio-status

    # Pre-install PyTorch Nightly if Python 3.13+ is detected
    if sys.version_info >= (3, 13):
        print("Detected Python 3.13+. Installing PyTorch Nightly...")
        !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

    # 3. Install Backend
    !pip install .

    # 4. Set API Key
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        print("Please enter your Gemini API Key:")
        os.environ["GEMINI_API_KEY"] = input()
# --- COLAB SETUP END ---

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the model strategy to use.
# @markdown *Note: Experimental models (2.0/2.5) may require billing/quota enablement.*

MODEL_STRATEGY = "Free Tier (Gemini 1.5 Flash)" # @param ["Free Tier (Gemini 1.5 Flash)", "Experimental (Gemini 2.0 Flash)", "Flash Lite (Gemini 2.0 Flash-Lite)", "Gemini 2.5 Flash (Experimental)", "Gemini 2.5 Pro (Experimental)"]

import os

# Map selection to actual model names
if MODEL_STRATEGY == "Free Tier (Gemini 1.5 Flash)":
    SELECTED_MODEL = "gemini-1.5-flash"
    print("✅ Using Free Tier Model: gemini-1.5-flash")
elif MODEL_STRATEGY == "Experimental (Gemini 2.0 Flash)":
    SELECTED_MODEL = "gemini-2.0-flash-exp"
    print("⚠️ Using Experimental Model: gemini-2.0-flash-exp (Quota Required)")
elif MODEL_STRATEGY == "Flash Lite (Gemini 2.0 Flash-Lite)":
    SELECTED_MODEL = "gemini-2.0-flash-lite-preview-02-05"
    print("⚠️ Using Preview Model: gemini-2.0-flash-lite-preview-02-05 (Quota Required)")
elif MODEL_STRATEGY == "Gemini 2.5 Flash (Experimental)":
    SELECTED_MODEL = "gemini-2.5-flash"
    print("🔥 Using Gemini 2.5 Flash: gemini-2.5-flash (Quota Required)")
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Experimental)":
    SELECTED_MODEL = "gemini-2.5-pro"
    print("🔥 Using Gemini 2.5 Pro: gemini-2.5-pro (Quota Required)")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

# Deep Research Agent: Core Workflow

This notebook demonstrates the core research workflow of the Deep Research Agent, including:
1. Query Decomposition
2. Web Search & RAG Ingestion
3. Evidence Auditing & Verification
4. Final Synthesis
5. Artifact Export

## 1. Setup & Dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add backend/src to path so we can import modules
sys.path.append(os.path.abspath("../backend/src"))

In [ ]:
from agent.deep_search_agent import DeepSearchAgent
from langchain_google_genai import ChatGoogleGenerativeAI
from agent.configuration import Configuration
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 2. Initialize Agent Components

In [ ]:
# Initialize LLM
# We use the SELECTED_MODEL from the config cell above
model_name = os.environ.get("ANSWER_MODEL", "gemini-1.5-flash")

try:
    print(f"Initializing LLM with model: {model_name}")
    llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    # Fallback for testing
    llm = None

if llm:
    # Initialize Agent
    # Note: DeepSearchAgent internally uses the Configuration object which reads from env vars
    agent = DeepSearchAgent(llm_client=llm)
    print("Agent initialized successfully.")
else:
    print("Failed to initialize Agent due to LLM error.")

## 3. Single Query Research Demo

In [ ]:
query = "Latest advances in renewable energy storage"
print(f"Starting research on: {query}")

if agent:
    final_answer = agent.research(query)
    print("\n=== FINAL ANSWER ===\n")
    print(final_answer)
else:
    print("Agent not initialized.")

## 4. RAG Verification Deep Dive

Inspect the internal state of the RAG system to verify ingestion and retrieval.

In [ ]:
if agent:
    # Inspect Doc Store
    print(f"Total documents indexed: {len(agent.rag.doc_store)}")

    # Sample a document
    if len(agent.rag.doc_store) > 0:
        doc_id = list(agent.rag.doc_store.keys())[0]
        print(f"\nSample Document ({doc_id}):")
        print(agent.rag.doc_store[doc_id])

In [ ]:
if agent:
    # Test Retrieval
    test_query = "battery cost reduction"
    results = agent.rag.retrieve(test_query, top_k=3)

    print(f"\nRetrieval results for '{test_query}':")
    for doc, score in results:
        print(f"- [{score:.2f}] {doc.content[:100]}...")

## 5. Export Research Artifacts

In [ ]:
if agent:
    state = agent.rag.export_state()
    print("RAG State Export:", state)

    # Retrieve all docs for export
    all_docs = agent.get_retrieved_documents()
    print(f"\nPrepared {len(all_docs)} documents for export.")